In [394]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import metrics   

from sklearn import set_config
set_config(display='diagram') # Useful for display the pipeline

In [395]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [396]:
#dataset_half_S = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow.csv', index_col=[0])

In [397]:
dataset.head(10)

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.000000,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.000000,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.00,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.000000,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.000000,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.000000,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.000000,0.0,0.0,NaN,1.488361

In [398]:
dataset.user.unique().tolist() #all users

['U12',
 'U1',
 'U8',
 'U7',
 'U10',
 'U13',
 'U6',
 'U11',
 'U3',
 'U2',
 'U4',
 'U9',
 'U5']

In [399]:
for name, group in dataset.groupby('user'):
    print(name,len(group))

U1 2470
U10 531
U11 167
U12 820
U13 90
U2 271
U3 331
U4 227
U5 17
U6 279
U7 397
U8 214
U9 79


In [400]:
dataset.target.unique().tolist() #all users

['Still', 'Car', 'Train', 'Bus', 'Walking']

In [401]:
# encode the target
#dataset.target = dataset.target.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':4})

In [402]:
dataset.columns

Index(['id', 'time', 'activityrecognition#0', 'activityrecognition#1',
       'android.sensor.accelerometer#mean', 'android.sensor.accelerometer#min',
       'android.sensor.accelerometer#max', 'android.sensor.accelerometer#std',
       'android.sensor.game_rotation_vector#mean',
       'android.sensor.game_rotation_vector#min',
       'android.sensor.game_rotation_vector#max',
       'android.sensor.game_rotation_vector#std',
       'android.sensor.gravity#mean', 'android.sensor.gravity#min',
       'android.sensor.gravity#max', 'android.sensor.gravity#std',
       'android.sensor.gyroscope#mean', 'android.sensor.gyroscope#min',
       'android.sensor.gyroscope#max', 'android.sensor.gyroscope#std',
       'android.sensor.gyroscope_uncalibrated#mean',
       'android.sensor.gyroscope_uncalibrated#min',
       'android.sensor.gyroscope_uncalibrated#max',
       'android.sensor.gyroscope_uncalibrated#std',
       'android.sensor.light#mean', 'android.sensor.light#min',
       'android.se

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [403]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5893 entries, 0 to 5892
Data columns (total 70 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   id                                               5893 non-null   int64  
 1   time                                             5893 non-null   float64
 2   activityrecognition#0                            0 non-null      float64
 3   activityrecognition#1                            5893 non-null   float64
 4   android.sensor.accelerometer#mean                5893 non-null   float64
 5   android.sensor.accelerometer#min                 5893 non-null   float64
 6   android.sensor.accelerometer#max                 5893 non-null   float64
 7   android.sensor.accelerometer#std                 5893 non-null   float64
 8   android.sensor.game_rotation_vector#mean         5094 non-null   float64
 9   android.sensor.game_rotation_v

In [404]:
dataset.describe()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,5094.000000,5093.000000,5396.000000,5396.000000,5396.000000,5.356000e+03,5173.000000,5173.000000,5173.000000,5172.000000,5094.000000,5094.000000,5094.000000,5094.000000,4381.000000,4381.000000,4381.000000,2124.000000,5399.000000,5399.000000,5399.000000,5370.000000,5400.000000,5400.000000,5400.000000,5383.000000,5094.000000,5094.000000,5094.000000,5087.000000,5396.000000,5396.000000,5396.000000,5373.000000,1918.000000,1918.000000,1918.000000,1128.000000,1119.000000,1119.000000,1119.000000,283.000000,5172.000000,5172.000000,5172.000000,5163.000000,1487.000000,1487.000000,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,0.771629,0.011876,9.806561,9.806530,9.806592,1.220395e-04,0.312536,0.089494,0.735042,0.189903,0.312807,0.125427,0.614961,0.174279,288.825545,254.140861,331.698065,85.357327,1.315971,0.863086,2.157592,0.602771,56.212508,54.267805,58.207109,1.772942,138.409066,136.955896,139.907248,2.038359,212.629637,199.930476,224.731494,12.488254,1016.203139,1016.189143,1016.218816,0.023097,6.117128,5.174263,6.797140,4.081700,0.765932,0.750273,0.779000,0.012977,10112.523523,10112.205783,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,0.251401,0.026647,0.001610,0.001709,0.001584,7.681037e-04,0.554585,0.206327,1.356240,0.342901,0.537925,0.235223,1.112451,0.322397,1761.573894,1361.427348,2379.942459,1052.057842,2.065444,1.493150,5.058223,1.136752,38.210015,37.902702,38.681821,2.938644,71.034674,70.764992,71.910605,5.423627,95.253318,98.160410,101.113269,31.517931,14.142150,14.146752,14.137338,0.081745,15.141369,15.34

In [405]:
dataset.isnull().sum()

id                                                    0
time                                                  0
activityrecognition#0                              5893
activityrecognition#1                                 0
android.sensor.accelerometer#mean                     0
android.sensor.accelerometer#min                      0
android.sensor.accelerometer#max                      0
android.sensor.accelerometer#std                      0
android.sensor.game_rotation_vector#mean            799
android.sensor.game_rotation_vector#min             799
android.sensor.game_rotation_vector#max             799
android.sensor.game_rotation_vector#std             800
android.sensor.gravity#mean                         497
android.sensor.gravity#min                          497
android.sensor.gravity#max                          497
android.sensor.gravity#std                          537
android.sensor.gyroscope#mean                       720
android.sensor.gyroscope#min                    

In [406]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-406-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000

In [407]:
dataset['acc*gyro_mean'] = dataset['accelerometer_mean'] * dataset['gyroscope_mean']
dataset['lin*speed_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean']


In [408]:
dataset = dataset[(['accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_uncalibrated_mean',
                    'acc*gyro_mean',
                    'gyroscope_std',
                    'linear_acceleration_mean',
                    'lin*speed_mean',
                    'rotation_vector_mean',
                    'rotation_vector_max',
                    'proximity_std',
                    'magnetic_field_uncalibrated_min',
                    'sound_min',
                    'user',
                    'target'
                 ])]

In [409]:
dataset.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,acc*gyro_mean,gyroscope_std,linear_acceleration_mean,lin*speed_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,user,target
0,9.758895,9.849411,0.014626,0.001651,0.016221,0.016197,0.000737,0.020978,0.000000,0.050413,0.056351,NaN,51.199707,NaN,U12,Still
1,7.707437,17.146631,1.775944,0.036326,0.039023,0.361049,0.020290,0.879220,14.541729,0.999981,0.999999,NaN,82.409890,89.065143,U12,Car
2,9.804817,9.849262,0.011199,0.001525,0.016302,0.014983,0.000713,0.034110,0.000000,0.610456,0.610456,NaN,55.501802,NaN,U12,Still
3,7.659674,12.304298,0.862553,0.036440,0.050759,0.352489,0.010553,1.488361,26.403360,0.998112,0.998112,NaN,95.664309,87.470377,U12,Car
4,8.965621,10.891645,0.504117,0.183202,0.265652,1.830824,0.098819,1.658308,14.924768,0.322242,0.378193,NaN,156.795909,89.770732,U1,Car


In [410]:
# split user = u12 & u3 as test/train
dataset_test = dataset[(dataset['user'] == 'U2')|(dataset['user'] == 'U6')|(dataset['user'] == 'U4')]
dataset_train = dataset[(dataset['user'] != 'U2')&(dataset['user'] != 'U6')&(dataset['user'] != 'U4')]
print(len(dataset_test))
print(len(dataset_train))

777
5116


In [411]:
dataset_train.sample(10)

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,acc*gyro_mean,gyroscope_std,linear_acceleration_mean,lin*speed_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,user,target
4691,5.789232,17.866860,4.224925,1.354170,1.281255,15.629269,0.753964,6.131868,21.461538,0.998820,1.000000,NaN,154.022317,89.711022,U1,Walking
5058,8.594723,10.577705,0.612402,0.280017,0.402294,2.697273,0.271088,0.775117,NaN,0.540474,0.540474,NaN,178.760566,81.017644,U7,Train
3030,8.787002,14.923951,2.242509,0.471855,0.074398,5.413900,0.454217,2.566938,NaN,0.995088,0.996242,NaN,143.443062,89.764526,U1,Bus
1815,7.165028,11.123493,1.170524,0.206484,0.199661,1.934882,0.163640,1.941134,NaN,0.978503,0.979465,NaN,130.262964,89.755490,U1,Bus
116,9.503682,9.710174,0.061950,0.006881,0.036020,0.066303,0.003261,0.115954,NaN,0.375896,0.376278,NaN,145.112385,89.150977,U1,Train
4363,4.980492,22.589287,3.976733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.494967,U8,Car
4763,9.526128,9.874026,0.082017,0.079860,0.097944,0.769501,0.031633,0.381811,0.000000,0.977175,0.980444,NaN,138.717120,89.741353,U1,Bus
4225,7.988747,11.889993,0.785698,0.171560,0.123016,1.684773,0.155454,0.395114,4.783381,0.689568,0.689568,NaN,173.189549,88.941299,U7,Bus
1776,7.877628,11.104245,1.068347,0.232780,0.186954,2.308178,0.191682,2.117070,15.878028,0.941392,0.942859,NaN,161.848335,89.806475,U1,Bus
1255,8.970947,9.333236,0.094522,0.079619,0.172685,0.723033,0.049754,0.872940,0.000000,0.994971,0.999214,NaN,72.227424,36.776982,U3,Walking


In [412]:
#dataset_train = dataset_train.fillna(0)

In [413]:
dataset_train.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,acc*gyro_mean,gyroscope_std,linear_acceleration_mean,lin*speed_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,user,target
0,9.758895,9.849411,0.014626,0.001651,0.016221,0.016197,0.000737,0.020978,0.000000,0.050413,0.056351,NaN,51.199707,NaN,U12,Still
1,7.707437,17.146631,1.775944,0.036326,0.039023,0.361049,0.020290,0.879220,14.541729,0.999981,0.999999,NaN,82.409890,89.065143,U12,Car
2,9.804817,9.849262,0.011199,0.001525,0.016302,0.014983,0.000713,0.034110,0.000000,0.610456,0.610456,NaN,55.501802,NaN,U12,Still
3,7.659674,12.304298,0.862553,0.036440,0.050759,0.352489,0.010553,1.488361,26.403360,0.998112,0.998112,NaN,95.664309,87.470377,U12,Car
4,8.965621,10.891645,0.504117,0.183202,0.265652,1.830824,0.098819,1.658308,14.924768,0.322242,0.378193,NaN,156.795909,89.770732,U1,Car


In [414]:
# drop user in test & split
dataset_train = dataset_train.drop(['user'], axis = 1)
dataset_test = dataset_test.drop(['user'], axis = 1)

In [415]:
num_vars = ['accelerometer_min',
            'accelerometer_max',
            'accelerometer_std',
            'gyroscope_mean',
            'gyroscope_uncalibrated_mean',
            'gyroscope_std',
            'acc*gyro_mean',
            'linear_acceleration_mean',
            'lin*speed_mean',
            'rotation_vector_mean',
            'rotation_vector_max',
            'proximity_std',
            'magnetic_field_uncalibrated_min',
            'sound_min'
            ]


In [416]:
# YOUR CODE HERE
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler
num_4_treeModels = pipeline.Pipeline(steps=[
    #('imputing', impute.SimpleImputer(missing_values=np.nan ,strategy='mean')),
    ('imputing', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler())
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputing', KNNImputer()),
                                                 ('scaler', RobustScaler())]),
                                 ['accelerometer_min', 'accelerometer_max',
                                  'accelerometer_std', 'gyroscope_mean',
                                  'gyroscope_uncalibrated_mean',
                                  'gyroscope_std', 'acc*gyro_mean',
                                  'linear_acceleration_mean', 'lin*speed_mean',
                                  'rotation_vector_mean', 'rotation_vector_max',
                                  'proximity_std',
                                  'magnetic_field_uncalibrated_min',
                                  'sound_min'])])

In [417]:


from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier



In [418]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}



tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputing',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['accelerometer_min',
                                                   'accelerometer_max',
                                                   'accelerometer_std',
                                                   'gyroscope_mean',
                                                   'gyroscope_uncalibrated_mean',
                                                   'gyroscope_std',
                                                   'acc*gyro_mean',
                                                   'linear_acceleration_mean',
                                                   'lin*speed_mean',
                                                   'rotation_vector_mean',
                                                   'rotation_vector_max',
                                                   'proximity_std',
                                                   'magnetic_field_uncalibrated_min',
                                                   'sound_min'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [419]:
# split 
x_train = dataset_train.drop(['target'], axis=1)
y_train = dataset_train.target
x_val = dataset_test.drop(['target'], axis=1)
y_val = dataset_test.target

In [ ]:
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:11:29] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.085996
0:	learn: 1.4735308	total: 53.9ms	remaining: 53.8s
1:	learn: 1.3748934	total: 85.5ms	remaining: 42.7s
2:	learn: 1.2874987	total: 135ms	remaining: 44.8s
3:	learn: 1.2166175	total: 169ms	remaining: 42.1s
4:	learn: 1.1593892	total: 207ms	remaining: 41.3s
5:	learn: 1.1120378	total: 258ms	remaining: 42.7s
6:	learn: 1.0657633	total: 287ms	remaining: 40.8s
7:	learn: 1.0215568	total: 336ms	remaining: 41.6s
8:	learn: 0.9889930	total: 360ms	remaining: 39.7s
9:	learn: 0.9489701	total: 388ms	remaining: 38.4s
10:	learn: 0.9147516	total: 440ms	remaining: 39.6s
11:	learn: 0.8882574	total: 509ms	remaining: 41.9s
12:	learn: 0.8646697	total: 546ms	remaining: 41.5s
13:	learn: 0.8435709	total: 588ms	remaining: 41.4s
14:	learn: 

158:	learn: 0.2359714	total: 8.19s	remaining: 43.3s
159:	learn: 0.2349931	total: 8.24s	remaining: 43.2s
160:	learn: 0.2337854	total: 8.3s	remaining: 43.2s
161:	learn: 0.2330767	total: 8.33s	remaining: 43.1s
162:	learn: 0.2321447	total: 8.38s	remaining: 43s
163:	learn: 0.2311499	total: 8.44s	remaining: 43s
164:	learn: 0.2299301	total: 8.46s	remaining: 42.8s
165:	learn: 0.2292804	total: 8.49s	remaining: 42.7s
166:	learn: 0.2285347	total: 8.53s	remaining: 42.5s
167:	learn: 0.2280204	total: 8.59s	remaining: 42.5s
168:	learn: 0.2274512	total: 8.64s	remaining: 42.5s
169:	learn: 0.2269021	total: 8.7s	remaining: 42.5s
170:	learn: 0.2262431	total: 8.74s	remaining: 42.4s
171:	learn: 0.2249185	total: 8.78s	remaining: 42.3s
172:	learn: 0.2243394	total: 8.84s	remaining: 42.3s
173:	learn: 0.2231288	total: 8.9s	remaining: 42.3s
174:	learn: 0.2224086	total: 8.94s	remaining: 42.1s
175:	learn: 0.2217200	total: 8.97s	remaining: 42s
176:	learn: 0.2211023	total: 9.01s	remaining: 41.9s
177:	learn: 0.2204276

318:	learn: 0.1455415	total: 15.8s	remaining: 33.7s
319:	learn: 0.1451413	total: 15.8s	remaining: 33.6s
320:	learn: 0.1444952	total: 15.9s	remaining: 33.6s
321:	learn: 0.1439478	total: 15.9s	remaining: 33.6s
322:	learn: 0.1435946	total: 16s	remaining: 33.6s
323:	learn: 0.1431998	total: 16.1s	remaining: 33.5s
324:	learn: 0.1428513	total: 16.1s	remaining: 33.5s
325:	learn: 0.1425388	total: 16.2s	remaining: 33.4s
326:	learn: 0.1423806	total: 16.2s	remaining: 33.3s
327:	learn: 0.1422196	total: 16.3s	remaining: 33.3s
328:	learn: 0.1420108	total: 16.3s	remaining: 33.2s
329:	learn: 0.1415389	total: 16.3s	remaining: 33.2s
330:	learn: 0.1409272	total: 16.4s	remaining: 33.1s
331:	learn: 0.1403660	total: 16.4s	remaining: 33s
332:	learn: 0.1399844	total: 16.5s	remaining: 33s
333:	learn: 0.1396899	total: 16.5s	remaining: 32.9s
334:	learn: 0.1391895	total: 16.5s	remaining: 32.8s
335:	learn: 0.1389852	total: 16.6s	remaining: 32.8s
336:	learn: 0.1384050	total: 16.6s	remaining: 32.7s
337:	learn: 0.1379

481:	learn: 0.0988466	total: 22.7s	remaining: 24.4s
482:	learn: 0.0987288	total: 22.8s	remaining: 24.4s
483:	learn: 0.0985820	total: 22.8s	remaining: 24.3s
484:	learn: 0.0984831	total: 22.8s	remaining: 24.3s
485:	learn: 0.0983542	total: 22.9s	remaining: 24.2s
486:	learn: 0.0982232	total: 22.9s	remaining: 24.2s
487:	learn: 0.0979085	total: 23s	remaining: 24.1s
488:	learn: 0.0977490	total: 23s	remaining: 24.1s
489:	learn: 0.0975587	total: 23.1s	remaining: 24s
490:	learn: 0.0974185	total: 23.1s	remaining: 23.9s
491:	learn: 0.0972413	total: 23.1s	remaining: 23.9s
492:	learn: 0.0970804	total: 23.2s	remaining: 23.8s
493:	learn: 0.0967425	total: 23.2s	remaining: 23.8s
494:	learn: 0.0964922	total: 23.3s	remaining: 23.7s
495:	learn: 0.0962743	total: 23.3s	remaining: 23.7s
496:	learn: 0.0961328	total: 23.3s	remaining: 23.6s
497:	learn: 0.0959987	total: 23.4s	remaining: 23.5s
498:	learn: 0.0956812	total: 23.4s	remaining: 23.5s
499:	learn: 0.0954878	total: 23.4s	remaining: 23.4s
500:	learn: 0.0953

646:	learn: 0.0721166	total: 28.8s	remaining: 15.7s
647:	learn: 0.0720222	total: 28.8s	remaining: 15.7s
648:	learn: 0.0718418	total: 28.9s	remaining: 15.6s
649:	learn: 0.0717893	total: 28.9s	remaining: 15.6s
650:	learn: 0.0716665	total: 28.9s	remaining: 15.5s
651:	learn: 0.0714633	total: 28.9s	remaining: 15.5s
652:	learn: 0.0713054	total: 29s	remaining: 15.4s
653:	learn: 0.0712508	total: 29s	remaining: 15.4s
654:	learn: 0.0709983	total: 29.1s	remaining: 15.3s
655:	learn: 0.0709277	total: 29.1s	remaining: 15.3s
656:	learn: 0.0707664	total: 29.1s	remaining: 15.2s
657:	learn: 0.0706922	total: 29.2s	remaining: 15.2s
658:	learn: 0.0705218	total: 29.2s	remaining: 15.1s
659:	learn: 0.0704391	total: 29.2s	remaining: 15.1s
660:	learn: 0.0703480	total: 29.3s	remaining: 15s
661:	learn: 0.0702723	total: 29.3s	remaining: 15s
662:	learn: 0.0701639	total: 29.3s	remaining: 14.9s
663:	learn: 0.0700687	total: 29.4s	remaining: 14.9s
664:	learn: 0.0699843	total: 29.4s	remaining: 14.8s
665:	learn: 0.069920

807:	learn: 0.0557917	total: 34.4s	remaining: 8.18s
808:	learn: 0.0556745	total: 34.5s	remaining: 8.14s
809:	learn: 0.0555820	total: 34.5s	remaining: 8.09s
810:	learn: 0.0555342	total: 34.5s	remaining: 8.05s
811:	learn: 0.0554683	total: 34.6s	remaining: 8.01s
812:	learn: 0.0554236	total: 34.7s	remaining: 7.98s
813:	learn: 0.0553840	total: 34.7s	remaining: 7.94s
814:	learn: 0.0552716	total: 34.8s	remaining: 7.89s
815:	learn: 0.0551945	total: 34.8s	remaining: 7.86s
816:	learn: 0.0551659	total: 34.9s	remaining: 7.82s
817:	learn: 0.0550942	total: 35s	remaining: 7.78s
818:	learn: 0.0549675	total: 35s	remaining: 7.74s
819:	learn: 0.0549234	total: 35s	remaining: 7.69s
820:	learn: 0.0548613	total: 35.1s	remaining: 7.65s
821:	learn: 0.0547640	total: 35.1s	remaining: 7.61s
822:	learn: 0.0547080	total: 35.2s	remaining: 7.56s
823:	learn: 0.0546405	total: 35.2s	remaining: 7.52s
824:	learn: 0.0545557	total: 35.2s	remaining: 7.47s
825:	learn: 0.0544566	total: 35.3s	remaining: 7.43s
826:	learn: 0.0543

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_val,pred))
print(classification_report(y_val, pred))